In [ ]:
!pip install gensim
!pip install nltk
import string
import nltk
import gensim
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import reuters
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('reuters')

import warnings

warnings.filterwarnings(action='ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Nouvelle section

In [ ]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:

class Preprocessor:
  def __init__(self, text:str):
    self.text = text

  def finish_preprocessing(self):
    return self.text

  def rest_text(self, new_text):
    self.text = new_text

  def lower(self):
    self.text = self.text.lower()
    return self

  def __validate_casting_into_set(self, set_):
    try:
      set_ = set(set_)
    except:
      raise TypeError('exceptions parameter should be turnable into a set. We suggest you provide an Iterable')

  def tokenization_1(self):
    self.text = word_tokenize(self.text)
    return self

  def embedding_1(self):
    vectorizer = CountVectorizer()
    self.text = vectorizer.fit_transform(self.text).toarray()
    return self

  def remove_punctuation(self, language, exceptions=set()):
    punctuations = set(string.punctuation)
    self.__validate_casting_into_set(exceptions)
    punctuations = punctuations - exceptions
    tokens = word_tokenize(self.text, language=language)
    tokens = [token for token in tokens if not token in punctuations]
    self.text = ' '.join(tokens)
    return self

   # remove white spaces like \n, \t, ... leaves blank space
  def remove_universal_newlines(self):
    paragraphs = self.text.splitlines()
    self.text = ' '.join(paragraphs)
    return self

  def remove_stopwords(self, language, exceptions=set()):
    stopwords = set(stopwords.words(language))
    self.__validate_casting_into_set(exceptions)
    stopwords = stopwords - exceptions
    tokens = word_tokenize(self.text, language=language)
    tokens = [token for token in tokens if not token in stopwords]
    self.text = ' '.join(tokens)
    return self

  def lemmatize_text(self, language, exceptions=set()):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(self.text, language = language)
    lemmatized_text = [lemmatizer.lemmatize(token) for token in tokens]
    self.text = ' '.join(lemmatized_text)
    return self

  # J'ai essayé de créer une deuxième fonction pour tokenization, lemmatization et vectorization pour qu'on puisse choisir laquelle on utilise

def tokenization_2(text):
    tokens = word_tokenize(text) #utiliser NLTK pour la tokenization
    stop_words = set(stopwords.words('french')) #Pour le prétraitement du texte
    tokens = [word for word in tokens if word.lower() not in stop_words]
    return tokens

def embedding_2(mot):
  if not isinstance(mot, str):
    raise TypeError("Le mot doit être une chaîne de caractères.")
  else:
    corpus = nltk.corpus.reuters
    sentences = corpus.words()  #obtention des phrases dans le corpus reuters
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
    #entrainement du modèle Word2Vec
    model = Word2Vec(sentences = tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)
    word_vector = model.wv[mot]   #Vectorisation d'un mot spécifique
    return word_vector

In [ ]:
#Test

test_1 = Preprocessor('Voici! un*@ et un +## ponctuation!! ')
tested_1 = test_1.remove_punctuation('french').finish_preprocessing()

print(tested_1)

Voici un et un ponctuation


In [ ]:
#Test_2
text_2 = 'ils mangent leurs nourritures et puis ils vont chez leurs tantes, jouent des footballs'
test_2 = Preprocessor(text_2)
tested_2 = test_2.lemmatize_text('french').finish_preprocessing()
print(tested_2)

il mangent leurs nourritures et puis il vont chez leurs tantes , jouent de football


In [ ]:
#test3

text_3 = "Tu vas où ce soir?"
test_3 = Preprocessor(text_3)
tested_3 = test_3.tokenization_1().finish_preprocessing()
tested_3


['Tu', 'vas', 'où', 'ce', 'soir', '?']

In [ ]:
#Test4
text_4 = "Tu VAS où CE soir?"
test_4 = Preprocessor(text_4)
tested_4 = test_4.lower().finish_preprocessing()
tested_4

'tu vas où ce soir?'

In [ ]:
#Test5
text_5 = "Tu VAS où CE soir?"
test_5 = Preprocessor(text_5)
tested_5 = test_5.lower().tokenization_1().embedding_1().finish_preprocessing()
print(tested_5)

[[0 0 0 1 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]]


In [ ]:
print(tokenization_2('Je vais au marché!'))

['vais', 'marché', '!']


In [ ]:
import spacy

In [ ]:
def lemmatize(text):   #lemmatization du texte en utilisant spacy
  nlp = spacy.load("en_core_web_sm")
  liste = nlp(text)
  lemmatized_tokens = [token.lemma_ for token in liste]
  lemmatized_tokens = ' '.join(lemmatized_tokens)
  return lemmatized_tokens


In [ ]:
text = 'ils mangent leurs nourritures et puis ils vont chez leurs tantes, jouent des footballs'
lemmatize(text)


'il mangent leur nourriture et puis il vont chez leurs tante , jouent des football'

In [ ]:
print(embedding_2('malade'))

KeyError: "Key 'malade' not present"

# Nouvelle section

# Nouvelle section

In [ ]:
!pip install langchain
!pip install sentence-transformers
!pip install elasticsearch
!pip install bitsandbytes

import torch
import os
import bitsandbytes
from threading import Thread
from typing import Optional
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,TextIteratorStreamer,pipeline
from langchain.llms import HuggingFacePipeline
from langchain.llms.base import LLM
from langchain.vectorstores import ElasticsearchStore
from langchain.vectorstores import ElasticVectorSearch
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate, LLMChain

def Load_document(text_path):  #fichier ".txt"
  loader = TextLoader(text_path)
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
  docs = text_splitter.split_documents(documents)
  return docs

def load_document(data_path):
  loader = PyPDFDirectoryLoader(data_path)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
  docs = text_splitter.split_documents(documents)
  return docs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.3 MB/s eta 0:00:00
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
docs = load_document("https://drive.google.com/drive/home")

In [ ]:
print(docs)

[]


In [ ]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
#vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")'''

#embed document chunks
#model_kwargs = {"device": "cuda"}
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)
embeddings = embedding.embed_documents(docs)
elastic_host = "9c9fd5219e1646eb98bbd90b7c552a78.us-central1.gcp.cloud.es.io"

elasticsearch_url = f"https://GnoseIA:Virginie971!@{elastic_host}:9243"
vectordb = ElasticVectorSearch(
                elasticsearch_url=elasticsearch_url,
                index_name="my-index",
                embedding=embeddings
            )

# specify the retriever
retriever = vectordb.as_retriever()

In [ ]:
print(embeddings)

[]


In [ ]:
# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model


In [ ]:
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer


In [ ]:
# Function for streaming llm ouput word by word and reduce perceived latency
def init_model(model, tokenizer):
    class CustomLLM(LLM):

        """Streamer Object"""

        streamer: Optional[TextIteratorStreamer] = None

        def _call(self, prompt, stop=None, run_manager=None) -> str:
            self.streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, Timeout=5)
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
            kwargs = dict(input_ids=inputs["input_ids"], streamer=self.streamer, max_new_tokens=500)
            thread = Thread(target=model.generate, kwargs=kwargs)
            thread.start()
            return ""

        @property
        def _llm_type(self) -> str:
            return "custom"

    llm = CustomLLM()

    return llm

In [ ]:
#!pip install mistral
#from mistral import Mistral
model_name = "mixtral:8x7b"
# load model
model = load_quantized_model(model_name)

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)

# specify stop token ids
stop_token_ids = [0]

OSError: Incorrect path_or_model_id: 'mixtral:8x7b'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2048,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

llm=init_model(model,tokenizer)

In [ ]:
# build conversational retrieval chain with memory (rag) using langchain
def create_conversation(query: str, chat_history: list) -> tuple:
    try:

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            memory=memory,
            get_chat_history=lambda h: h,
        )

        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))
        return '', chat_history


    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history